In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip uninstall keras -y
! pip install segmentation-models

Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 51kB 3.2MB/s 


In [2]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/14241/862020/compressed/train_images.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603871481&Signature=JFxqrX886uWOG0f3ALYABbTLDnxWhbZxYVVuiIX%2B8czGp1jEvE7qlwdz5b0q334B%2F8eEJkEwN3XWaBWSgMAAhe3N49VSQzPEtW02fVWccvnaB4OaC1xbqSJamWsd0nqRG0KJHg%2FfzE0zFFTtx7gpuKrFHcAjjvjPfS7bfiP4GyFHU9SSZNVIUPF3NNFh%2FvodjYVkHAgS8O5NZ2RnFzZ71Fi1Uy9gNe0MU18RqiSO4pJPKfanpVKk3cUel7%2Bo6uYbeHxoyyqAHDsiMpy5uK3nMA%2FP7yn2YIHm9gaXM8g2H2JfyRS%2BjO3JUE2CIVtL2mhp7UE5qnKM5wwZjmBy9haDtA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain_images.zip" -c -O 'train_images.zip'

--2020-10-25 07:51:33--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/14241/862020/compressed/train_images.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1603871481&Signature=JFxqrX886uWOG0f3ALYABbTLDnxWhbZxYVVuiIX%2B8czGp1jEvE7qlwdz5b0q334B%2F8eEJkEwN3XWaBWSgMAAhe3N49VSQzPEtW02fVWccvnaB4OaC1xbqSJamWsd0nqRG0KJHg%2FfzE0zFFTtx7gpuKrFHcAjjvjPfS7bfiP4GyFHU9SSZNVIUPF3NNFh%2FvodjYVkHAgS8O5NZ2RnFzZ71Fi1Uy9gNe0MU18RqiSO4pJPKfanpVKk3cUel7%2Bo6uYbeHxoyyqAHDsiMpy5uK3nMA%2FP7yn2YIHm9gaXM8g2H2JfyRS%2BjO3JUE2CIVtL2mhp7UE5qnKM5wwZjmBy9haDtA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain_images.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 108.177.127.128, 2a00:1450:4013:c08::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248443507 (1.2G) [application/zip]
Saving to: ‘train_images.zip’

t

In [ ]:
!mkdir 'images'
!unzip train_images.zip -d 'images'

In [4]:
import pandas as pd
train2 = pd.read_pickle('/content/gdrive/My Drive/train2')

In [5]:
train2.head()

,ImageId,e1,e2,e3,e4,count
0,0002cc93b.jpg,[29102 12 29346 24 29602 24 29858 24 30114 24 ...,,,,1
1,0007a71bf.jpg,,,[18661 28 18863 82 19091 110 19347 110 19603 1...,,1
2,000a4bcdd.jpg,[37607 3 37858 8 38108 14 38359 20 38610 25 38...,,,,1
3,000f6bf48.jpg,,,,[131973 1 132228 4 132483 6 132738 8 132993 11...,1
4,0014fce06.jpg,,,[229501 11 229741 33 229981 55 230221 77 23046...,,1


In [9]:
import numpy as np

y = np.empty((256,1600,4),dtype=np.int8)   
for j in range(4):
  y[:,:,j] = rle2mask(train2['e'+str(j+1)].iloc[0])

NameError: ignored

In [ ]:
y.shape

In [25]:
def mask(encoded_pixels):
    counts=[]
    mask=np.zeros((256*1600), dtype=np.uint8) 
    pre_mask=np.asarray([int(point) for point in encoded_pixels.split()])
    for index,count in enumerate(pre_mask):
        if(index%2!=0):
            counts.append(count)
    i=0
    for index,pixel in enumerate(pre_mask):
        if(index%2==0):
            if(i==len(counts)):
                break
            mask[pixel:pixel+counts[i]]=1
            i+=1
    mask=np.reshape(mask,(1600,256))
    mask=cv2.resize(mask,(HEIGHT,WIDTH)).T
    return mask


def rle(img):
    '''
    img: numpy array, 1 -> mask, 0 -> background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] 
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [11]:
train = pd.read_csv("/content/gdrive/My Drive/train.csv")

In [12]:
train.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [13]:
train.ClassId.value_counts()

3    5150
1     897
4     801
2     247
Name: ClassId, dtype: int64

In [18]:
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import rotate

HEIGHT=256
WIDTH=1600

new = pd.DataFrame(columns=['ImageId','ClassId','EncodedPixels'])

for i in tqdm(range(len(train))):

  ClassId = train.ClassId.iloc[i]

  if ClassId == 1 or ClassId == 4:
    # fliplr
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])

    img = np.fliplr(img)
    x = np.fliplr(x)
    img = cv2.resize(img, (1600,256), interpolation=cv2.INTER_LINEAR)
    
    plt.imsave('images/lr'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'lr'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #flipud
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])

    img = np.flipud(img)
    x = np.flipud(x)
    img = cv2.resize(img, (1600,256), interpolation=cv2.INTER_LINEAR)
    
    plt.imsave('images/ud'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'ud'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #fliplr+flipud
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])

    img = np.flipud(img)
    x = np.flipud(x)
    img = np.fliplr(img)
    x = np.fliplr(x)
    img = cv2.resize(img, (1600,256), interpolation=cv2.INTER_LINEAR)
    
    plt.imsave('images/lrud'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'lrud'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

  elif ClassId == 2:
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])

    #fliplr
    img = np.fliplr(img)
    x = np.fliplr(x)
    img = cv2.resize(img, (1600,256), interpolation=cv2.INTER_LINEAR)
    
    plt.imsave('images/lr'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'lr'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #flipud
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])

    img = np.flipud(img)
    x = np.flipud(x)
    img = cv2.resize(img, (1600,256), interpolation=cv2.INTER_LINEAR)
    
    plt.imsave('images/ud'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'ud'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #fliplr+flipud
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])

    img = np.flipud(img)
    x = np.flipud(x)
    img = np.fliplr(img)
    x = np.fliplr(x)
    img = cv2.resize(img, (1600,256), interpolation=cv2.INTER_LINEAR)
    
    plt.imsave('images/lrud'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'lrud'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #rotate 2
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])

    img = rotate(img,2)
    x = rotate(x,2)
    
    plt.imsave('images/rot2'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'rot2'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #rotate 2 fliplr
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])
    
    x = np.fliplr(x)
    img = np.fliplr(img)
    x = rotate(x,2)
    img = rotate(img,2)

    plt.imsave('images/rot2lr'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'rot2lr'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #rotate 2 flipud
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])
    
    x = np.flipud(x)
    img = np.flipud(img)
    x = rotate(x,2)
    img = rotate(img,2)

    plt.imsave('images/rot2ud'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'rot2ud'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

    #rotate 2 flipud fliplr
    img = cv2.imread('images/'+str(train.ImageId.iloc[i]))
    x = mask(train.EncodedPixels.iloc[i])
    
    x = np.fliplr(x)
    img = np.fliplr(img)
    x = np.flipud(x)
    img = np.flipud(img)
    x = rotate(x,2)
    img = rotate(img,2)

    plt.imsave('images/rot2lrud'+train.ImageId.iloc[i],img)
    x = rle(x)
    new_row = {'ImageId':'rot2lrud'+train.ImageId.iloc[i], 'ClassId':ClassId, 'EncodedPixels':x}

    new = new.append(new_row,ignore_index=True)

100%|██████████| 7095/7095 [08:39<00:00, 13.67it/s]


In [19]:
new.head()

,ImageId,ClassId,EncodedPixels
0,lr0002cc93b.jpg,1,216139 5 216395 14 216651 23 216907 28 217163 ...
1,ud0002cc93b.jpg,1,28998 12 29254 24 29510 24 29766 24 30022 24 3...
2,lrud0002cc93b.jpg,1,216240 5 216487 14 216734 23 216985 28 217241 ...
3,lr000a4bcdd.jpg,1,291421 6 291667 16 291840 1 291912 27 292096 1...
4,ud000a4bcdd.jpg,1,37398 3 37654 8 37910 14 38165 20 38421 25 386...


In [20]:
new = new.append(train)

In [21]:
new.ClassId.value_counts()

3    5150
1    3588
4    3204
2    1976
Name: ClassId, dtype: int64

In [37]:
new.to_pickle('/content/gdrive/My Drive/new')

In [38]:
!zip -r new_images.zip 'images'

Streaming output truncated to the last 5000 lines.
  adding: images/ed73c3216.jpg (deflated 1%)
  adding: images/a817da1df.jpg (deflated 0%)
  adding: images/4e2c9230d.jpg (deflated 0%)
  adding: images/3c79d2e28.jpg (deflated 0%)
  adding: images/0bd505b1b.jpg (deflated 0%)
  adding: images/lr8ac4ba80d.jpg (deflated 1%)
  adding: images/lrcbb659739.jpg (deflated 8%)
  adding: images/6355bcd53.jpg (deflated 0%)
  adding: images/2dfedfe3c.jpg (deflated 0%)
  adding: images/udbbde78fe1.jpg (deflated 1%)
  adding: images/16d9c0a30.jpg (deflated 1%)
  adding: images/6505eb32e.jpg (deflated 1%)
  adding: images/dab4aae2e.jpg (deflated 2%)
  adding: images/9aec868be.jpg (deflated 0%)
  adding: images/ude52f2f2de.jpg (deflated 1%)
  adding: images/097e28492.jpg (deflated 3%)
  adding: images/c92192495.jpg (deflated 0%)
  adding: images/54a1df397.jpg (deflated 2%)
  adding: images/6e71531ae.jpg (deflated 0%)
  adding: images/164ed8257.jpg (deflated 1%)
  adding: images/39fe944cc.jpg (deflated 

In [39]:
!cp '/content/new_images.zip' '/content/gdrive/My Drive'